## NetworkFile from ICL Code, PNExtract To ARM input

In [2]:
import numpy as np
from pathlib import Path
import pandas as pd
import os

def Statoil_load(path, prefix):
    network = {}
    
    path = Path(path)
    filename = Path(path.resolve(), prefix + '_link1.dat')   
    with open(filename, mode='r') as f:
        link1 = pd.read_csv(filepath_or_buffer=f,
                              header=None,
                              skiprows=1,
                              sep=' ',
                              skipinitialspace=True,
                              index_col=0)
        link1.columns = ['throat.pore1', 'throat.pore2', 'throat.radius',
                             'throat.shape_factor', 'throat.total_length']
    network['throat_connc'] = np.vstack((link1['throat.pore1'] - 1, link1['throat.pore2'] - 1)).T 
    network['throat_len'] = np.vstack(link1['throat.total_length'])
#----------------------------------------------------------------------------#    


    filename = Path(path.resolve(), prefix + '_node2.dat')
    with open(filename, 'r') as f:
        node2 = pd.read_csv(f,header=None, 
                            sep=' ',
                            skipinitialspace=True,
                            index_col=0)
        node2.columns=['pore_volume', 'pore_radius', 
                       'pore_shape_factor', 'pore_clay_volume']
        network['pore_radius'] = np.vstack(node2['pore_radius'])
        
#----------------------------------------------------------------------------#  


    filename = Path(path.resolve(), prefix + '_node1.dat')
    cnnct_pore = [];
    cnnct_throat = [];
    with open(filename, 'r') as f:
        info = f.readline().split()
        nor = int(info[0])
        array = np.ndarray([nor,6])
        for i in range(nor):
            row = f.readline().replace('\t',' ').replace('\n',' ').split();
            array[i,:] = row[0:6]
            noc = int(row[4]);
            
            cnnct_pore.append(row[5:5+noc]) #node number starts with 1 here
            cnnct_throat.append(row[5+noc+2:])
            
        node1 = pd.DataFrame(array[:,[1,2,3,4]]);
        node1.columns = ['pore.x_coord', 'pore.y_coord', 'pore.z_coord',
                         'pore.coordination_number']
        
    network['pore_cords'] = np.vstack((node1['pore.x_coord'],node1['pore.y_coord'], node1['pore.z_coord'])).T
    network['pcn'] = np.vstack(node1['pore.coordination_number']) #its returning 2D array
    network['connc_p2p'] = np.vstack(cnnct_pore) #list of connected nodes row(1) and connected pores row(2)
    network['connc_p2t'] = np.vstack(cnnct_throat)
    #----------------------------------------------------------------------------
    
    
    #Trimming the inlet and outlet throats and modifing the cordination number
    #Outlets
    trim_net_out = np.where(np.any(network['throat_connc'] == -1, axis = 1))[0]
    outlets = network['throat_connc'][trim_net_out, 1]
    remained_net_connc = np.delete(network['throat_connc'], trim_net_out, axis = 0)
    remained_net_len = np.delete(network['throat_len'], trim_net_out, axis = 0)
    
    #Inlets
    trim_net_in = np.where(np.any(remained_net_connc == -2, axis = 1))[0]
    inlets = remained_net_connc[trim_net_in, 1]
    
    final_net_connc = np.delete(remained_net_connc, trim_net_in, axis = 0)
    final_net_len = np.delete(remained_net_len, trim_net_in, axis = 0)
    
    network['throat_connc'] = np.vstack(final_net_connc);
    network['throat_len'] = np.vstack(final_net_len);
    network['inlets'] = np.vstack(inlets);
    network['outlets'] = np.vstack(outlets);
    #===========================================================================
    
    """need to review in case if connected more then on inlet or outlet pore"""
    
    network['pcn'][outlets] = network['pcn'][outlets] - 1;
    network['pcn'][inlets] = network['pcn'][inlets] - 1; 
    
    net_pore_wout_i = np.where(network['connc_p2p'] == '-1')
    #tmp = np.delete(net_pore_wout_i,)
    #print((net_pore_wout_i[0]))
#     net_throat_wout_i = np.where(np.any(network['list_connection'] == -1, axis = 1))
    
#     network['list_connection'] = np.vstack((net_wout_i, net_wout_o))
    
#     for i in outlets:
#         tmp = network['list_connection'][0,i].index('0')
#         del network['list_connection'][0,i][tmp] #delete pore
#         del network['list_connection'][1,i][tmp] #delete throat
#     for i in inlets:
#         tmp = network['list_connection'][0,i].index('-1')
#         del network['list_connection'][0,i][tmp] #delete pore
#         del network['list_connection'][1,i][tmp] #delete throat
    
    return network;


def StatoilToARM(path, prefix, network):
    """The input format of network consists
    Throats(cylinders) and Pores(spheres).
    In ARM, Pores are cylindrical and there are no throats
    So network topology is based on the thoats as they are connection between pores"""
    'Input: Statoil generated network from ICL pn-extract code '
    ARM_pore = {}
    ARM_pore_info =  distributed_volume(net = network)   # array of calculated radius of ARM_pore and pore_cordination_number
    pore_data = pd.DataFrame(data = ARM_pore_info, columns = ['ARM_pore_radius', 'ARM_pcn'])
    Q  = 0 * np.ndarray((len(pore_data['ARM_pcn']),1))
    ARM_pore['pore_data'] = np.vstack(( pore_data['ARM_pore_radius'].round(10), net['throat_len'].T.round(10), pore_data['ARM_pcn'], Q.T))
    ARM_node_coords = network['pore_cords'];
    #Node data
    node_pcn = network['pcn'][:,0];

    
    #setting up inlet and outlet status
    node_type = np.zeros(len(ARM_node_coords));
    for i in range(len(ARM_node_coords)):
        if np.any(network['inlets'] == i):
            node_type[i] = 1; #Inlet  = -1
        elif np.any(network['outlets'] == i):
            node_type[i] = -1; #outlet = 1
    
    #Checking directory exists or not to write the data
    path = Path(path)
    dir = Path(path.resolve(), prefix);
    if not os.path.exists(dir):
        os.makedirs(dir)
           
    filename = Path(dir.resolve(),prefix+'_pores_from_Rishabh')
    file = open(filename,'+w')
    
    head = ['#------------------------- ARM input file with pore information-------------------------#\n'] 
    head.append('# name \t d \t l \t b \t q \t Da_local \t G_local \n');
    head.append('# ---------------------------------------------------------------\n');
    file.write(''.join(head))
    
    data = pd.DataFrame(ARM_pore['pore_data'].T, dtype=float)
    data.to_csv(file, header=None,sep = '\t')
    file.close();
    #-------------------------------------------------------------------
    
    filename = Path(dir.resolve(),prefix+'_net_from_Rishabh')
    file = open(filename,'+w')

    head =     ['#--------------------------------ARM input file with network information--------------------------------------#\n'] 
    head.append('NN = ' + str(len(ARM_node_coords)) + '\n')
    head.append('NP = ' + str(len(pore_data['ARM_pore_radius'])) + '\n')
    head.append('NG = ' + str(0) + '\n')
    head.append('Nx = ' + str(0) + '\n')
    head.append('Ny = ' + str(0) + '\n')
    head.append('Ni = ' + str(len(network['inlets'])) + '\n')
    head.append('No = ' + str(len(network['outlets'])) + '\n')
    head.append('#    name                     position(x,y,z)    type      b      list of neighbors: (node_name, pore_name)\n')
    head.append('# -----------------------------------------------------------------------------------------------------------#\n')
    file.write(''.join(head));               
    
    data = [];
    for i in range(len(ARM_node_coords)):
        data.append(str(i) + '\t' +str(ARM_node_coords[i,0]) + '\t' + str(ARM_node_coords[i,1]) + '\t' + str(ARM_node_coords[i,2]) + '\t' + str(node_type[i]) + '\t');
        data.append(str(node_pcn[i]) + '\t')
        for j in range(int(node_pcn[i])):
                       data.append('( '+ str(int(net['list_connection'][0,i][j]))+','+(net['list_connection'][1,i][j]) + ' )    ')
        data.append('\n')
    
    file.write(''.join(data))
    file.close();    
    

def distributed_volume(net):
    """
    This function distribute the volume of spherical pores(PnExtract philosophy) among the connections
    Input:
            Network obj
            
    Output:
            returns array[np.throat, 2] of 
                col1: radius of cylinderical pores(ARM philosophy), 
                col2: ARM_pore_cordination_number
            
            Radius of ARM_pore is equal to 
                                                    Volume of connected Spherical pores
            Volume of cylinderical ARM_pore = Sum( ----------------------------------------)
                                                        (pore_coordination_number)
                                                (4.pi.r1^3)                   (4.pi.r2^3)
            pi* r_throat^2 * l  =      ------------------------- +    ----------------------------
                                        (3 * pore_cordination no.)      (3 * pore_cordination no.)
    """
    
    pcn = net['pcn']
    data = np.zeros((len(net['throat_connc']),2));
    #Calculating distributed Volume for each ARM_pore   
    for i in range(len(net['throat_connc'])):
        Pore1, Pore2 = net['throat_connc'][i]
        V1 = (4.0* np.pi * pow(net['pore_radius'][Pore1],3)/(3.0 * pcn[Pore1]))
        V2 = (4.0* np.pi * pow(net['pore_radius'][Pore2],3)/(3.0 * pcn[Pore2]))
        data[i,0] = np.sqrt((V1 + V2)/(2 * np.pi * net['throat_len'][i]))
        data[i,1] = (pcn[Pore1] + pcn[Pore2] - 2);
        
    return data
    
    
    
def WriteVTK(net):
    f = open('net.vtk','w+')
    f.write("# vtk DataFile Version 2.0\n Network example\nASCII\nDATASET POLYDATA\n")
    cords = net['pore_cords']
    const = 2 + np.ndarray([len(net['throat_connc']),1])
    connc = np.concatenate((const, net['throat_connc']), axis = 1)
    row = np.shape(cords)[0]
    throat = len(connc)
    f.write("POINTS %d float\n" %row)
    np.savetxt(f, cords)
    f.write("LINES %d %d\n" %(throat, 3*throat) )
    np.savetxt(f,connc.astype(int), fmt = '%i')
    f.close();

In [3]:
#load
case = 'test'
path = 'PNExtract_network/' + case #+ '2D'
net = Statoil_load(path = path,prefix=case)

In [9]:
print(net['connc_p2p'])
print("=================================")

tmp = np.where(np.any(net['connc_p2p'] == '-1', axis = 1));
print(tmp)
print(np.delete(net['connc_p2p'][1], tmp, axis = 1))

[['0' '2' '4' '5']
 ['0' '1' '3' '6']
 ['0' '2' '4' '7']
 ['0' '3' '1' '8']
 ['-1' '6' '1' '8']
 ['-1' '5' '2' '7']
 ['-1' '6' '3' '8']
 ['-1' '7' '4' '5']]
(array([4, 5, 6, 7]),)


AxisError: axis 1 is out of bounds for array of dimension 1

In [91]:
#convert and write
outpath = 'PnextractToARM/'
StatoilToARM(path = outpath, prefix = case, network = net)

IndexError: string index out of range